In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/Machine_Learning_Project/'

import pandas as pd
import numpy as np
import gc
from pandas.tseries.offsets import MonthEnd, MonthBegin

import datetime 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tqdm import tqdm
import io
from numpy.random import default_rng
from sklearn.utils import resample
from dateutil.relativedelta import relativedelta

# Data Preparation

## (1) Memory

To reduce memory space of each dataframe

In [ ]:
def reduce_memory_usage(df, skip_cols=['permno', 'Date']):

  """
  This function is to reduce the memory usage for each dataframe by downcasting the data type

  Reference: T.A

  """

  for col in df.columns:
        if col in skip_cols:
            continue
        col_type = df[col].dtype

        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(
                        np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(
                        np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(
                        np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(
                        np.int64).max:
                    df[col] = df[col].astype(np.int64)

            elif str(col_type)[:3] == "flo":
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(
                        np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(
                        np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            pass
            #df[col] = df[col].astype('category')
  return df

## (2) Macroeconomic variables

In [ ]:
#Read this into standard memory so it doesn't take up RAM in function. 

#Current Market Rates
market_rates_history = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Machine Learning/Machine_Learning_Project/PMMS_history.csv', usecols=["date", "pmms30"])
market_rates_history["date"] = pd.to_datetime(market_rates_history['date'].astype(str),  format='%m/%d/%Y')
market_rates_history.drop(market_rates_history.loc[market_rates_history["date"] < datetime.datetime(1999, 1, 1)].index, inplace = True)
market_rates_history = market_rates_history.resample('M', on='date').mean()
market_rates_history["date"] = market_rates_history.index.get_level_values(0)+ pd.tseries.offsets.MonthBegin(-1)
market_rates_history.reset_index(drop=True, inplace=True)

#HPI
hpi_history = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Machine Learning/Machine_Learning_Project/fmhpi_master_file.csv', usecols=["Year", "Month", "GEO_Name", "Index_SA"])
hpi_history["date"] = pd.to_datetime(hpi_history["Year"].astype(str) + '/' + hpi_history["Month"].astype(str) + "/1")
hpi_history["date"] = pd.to_datetime(hpi_history['date'].astype(str),  format='%Y/%m/%d')
hpi_history.drop(columns=["Year", "Month"], inplace=True)
hpi_history.drop(hpi_history.loc[hpi_history["date"] < datetime.datetime(1999, 1, 1)].index, inplace = True)
hpi_history.reset_index(drop=True, inplace=True)

#Unemployment
unemployment_history = pd.read_excel(r'/content/drive/My Drive/Colab Notebooks/Machine Learning/Machine_Learning_Project/unemployment_bystates.xlsx', usecols=["Year", "Month", "GEO_Name", "unemployment_rate"])
unemployment_history = unemployment_history.astype({"Year":"int", "Month":"int"})
unemployment_history["date"] = pd.to_datetime(unemployment_history["Year"].astype(str) + '/' + unemployment_history["Month"].astype(str) + "/1")
unemployment_history["date"] = pd.to_datetime(unemployment_history['date'].astype(str),  format='%Y/%m/%d')
unemployment_history.drop(columns=["Year", "Month"], inplace=True)
unemployment_history.drop(unemployment_history.loc[unemployment_history["date"] < datetime.datetime(1999, 1, 1)].index, inplace = True)
unemployment_history.dropna(axis=0, inplace=True)
unemployment_history.reset_index(drop=True, inplace=True)

#Merge all together
macro_variables = pd.merge(left = hpi_history,
                           right = unemployment_history,
                           how = "inner", on = ["date", "GEO_Name"],
                           validate = "1:1")
macro_variables = pd.merge(left = macro_variables,
                           right = market_rates_history,
                           how = "left", on = "date",
                           validate = "m:1")

In [ ]:
#macro_variables

## (3) Single Loan-Level Dataset

In [ ]:
dir = '/content/drive/MyDrive/Colab Notebooks/Machine Learning/Machine_Learning_Project/'

Extract columns to be used in the whole dataset

In [ ]:
origination_col_names = {0: 'fico_ori',
                         1: 'first_pmt_date',
                         7: 'occupancy_stat',
                         9: 'DTI_ori',
                         10: 'UPB_ori',
                         11: 'LTV_ori',
                         12: 'origin_interest',
                         14: 'ppm_flag',
                         16: 'state',
                         19: 'loan_seq_num',
                         20: 'purpose'}

performance_col_names = {0:'loan_seq_num',
                         1: 'month',
                         2: 'current_UPB',
                         4: 'loan_age',
                         5: 'time_to_maturity',
                         10: 'current_interest'}

### 3.1. Open files

In [ ]:
#Function to check duplicate

def check_duplicates(temp_file):
    if temp_file.duplicated().max() == True:
        temp_file = temp_file.drop_duplicates()
        #print("Duplicate rows discarded.")
    return temp_file

In [ ]:
#Function to set index

def set_index(dataframe, *fields):
    temp_list = []
    for field in fields:
        temp_list.append(field)
    dataframe.set_index(keys=temp_list, inplace = True, verify_integrity = True)
    dataframe.sort_index(inplace = True)

In [ ]:
def open_origination(file_name, dir = dir, col_numbers = [0, 1, 7, 9, 10, 11, 12, 16, 19, 20], col_names = origination_col_names):
    """"
    This function assumes that drive is already mounted. Change data directory to the correct folder.
  
    Arguments:
    - file_name [string] the name of the file, should be in format 'historical_data_YYYYQ#.txt', can pass this using below functions for file names
    Return:
    - temp_file [DataFrame] the completed DataFrame after checking for duplicates
    """

    temp_file = pd.read_csv(dir + file_name, 
                            sep="|", header=None, usecols= col_numbers).rename(columns = origination_col_names)
  
    temp_file.dropna(inplace = True)
    check_duplicates(temp_file)

    return temp_file

In [ ]:
def open_monthly_performance(file_name, dir = dir, col_numbers = [0, 1, 2, 4, 5, 10], col_names = performance_col_names):
    """"
    This function assumes that drive is already mounted. Change data directory to the correct folder.
  
    Arguments:
    - file_name [string] the name of the file, should be in format 'historical_data_time_YYYYQ#.txt', can pass this using below functions for file names
    Return:
    - temp_file [DataFrame] the completed DataFrame after checking for duplicates
    """

    temp_file = pd.read_csv(dir + file_name, 
                            sep="|", header=None, usecols= col_numbers).rename(columns = col_names)
  
    temp_file.dropna(inplace=True)
    check_duplicates(temp_file)

    return temp_file

In [ ]:
def file_name_ori(year, quarter):
    """
    Returns the file name for the Origination file in the correct format

    year, quarter = The year and quarter correspondng to the data files to be opened. year in [1999, 2021]; quarter in [1, 4]
    """
    return "historical_data_" + str(year) + "Q" + str(quarter) + ".txt"
  
def file_name_mon_per(year, quarter):
    """
    Returns the file name for the Monthly Performance file in the correct format

    year, quarter = The year and quarter correspondng to the data files to be opened. year in [1999, 2021]; quarter in [1, 4]
    """
    return "historical_data_time_" + str(year) + "Q" + str(quarter) + ".txt"


### 3.2. Merge columns and sample data

The `create_quarterly` function below will
- load all the data associated with the loans originated within the specified quarter and year
- sample data by the loan number sequence (`loan_seq_num`)
- calculate the needed variables (e.g., `rolling_incentive`) for chosen loans
- merge together the features of loans, the macroeconomic variables, and the calculated variables


__Sampling Method__

Consider the structure of the dataset:
- Each data file from Freddie Mac contains all the data of all the loans originated within the quarter of the year that the file is named after
- Suppose that there are $N$ loans originated in quarter $Q$ of year $YYYY$. Each loan is charaterized by a unique loan sequence number. Each record (i.e, each row) in the dataset reports the data associated with each loan at point in time (i.e., each month). 
- The total number of records is: <br> $\large number\ of\ rows = \Sigma^{N}_{i = 1} T_i$
<br> where $T_i$ is the number of monthly records of $loan_i$

After calculating all the needed variables, we sample __100,000__ random loan at any time t for each quarterly file

In [ ]:
def create_quarterly(year, quarter):
    """
    Imports quarterly data from external source, merge together Origination, Monthly Performance, and Macro Variables
    Randomly sample loans, drop all other data points, calculate needed variables.

    Arguments:
    - year, quarter = The year and quarter correspondng to the data files to be opened. year in [1999, 2021]; quarter in [1, 4]

    Returns 
    - quarterly_all [DataFrame] with 100,000 randomly sampled data points 
    """


    #----(1) Load "Monthly Performance" data for the year and quarter specified -----
    #Base DataFrame from Monthly Performance file
    quarterly_all = open_monthly_performance(file_name_mon_per(year, quarter))
    quarterly_all["month"] = pd.to_datetime(quarterly_all['month'].astype(str), format='%Y%m') 
    quarterly_all["month"] = quarterly_all["month"] + pd.tseries.offsets.MonthBegin(-1)
    reduce_memory_usage(quarterly_all)
    
    
    #-----(2) Load "Origination" data to merge with the "Base DataFrame"----------
    temp_ori = open_origination(file_name_ori(year, quarter))
    reduce_memory_usage(temp_ori)
    quarterly_all = pd.merge(left = quarterly_all,
                       right = temp_ori,
                       how = "left", on = "loan_seq_num",
                       validate = "m:1")
    
    del temp_ori
    gc.collect()
  

    #------(3) Add macro variables to dataset ---------------------------------
    #use only this for now to increase computing speed and reduce memory usage
    quarterly_all = pd.merge(left = quarterly_all,
                       right = macro_variables,
                       how = "left", 
                       left_on = ["month", "state"],
                       right_on = ["date", "GEO_Name"],
                       validate = "m:1")
    
    #Downcast data before calculations
    reduce_memory_usage(quarterly_all)
    
    #----(4) Calculate Interest Incentive and Rolling Interest Incentive--------
    quarterly_all["interest_incentive"] = quarterly_all["current_interest"] - quarterly_all["pmms30"]
    quarterly_all['rolling_incentive'] = quarterly_all.groupby('loan_seq_num')['interest_incentive'].transform(lambda x: x.rolling(24, 1).mean())
  
    quarterly_all["previous_UPB"] = quarterly_all.groupby("loan_seq_num")["current_UPB"].shift(1)
    
    quarterly_all.dropna(inplace = True, axis = 0)
    
    quarterly_all = resample(quarterly_all, replace=False, n_samples=100000, random_state=21)

    #Housekeeping for speed and less RAM usage
    set_index(quarterly_all, "loan_seq_num", "month")

    
    #------(5) Calculate remaining variables -----------------------------------
  
    quarterly_all['SATO'] = quarterly_all['origin_interest'] - quarterly_all.groupby('loan_seq_num')['pmms30'].transform(lambda x: x.iloc[0])
  
    quarterly_all['HPI_change'] = quarterly_all.groupby('loan_seq_num')['Index_SA'].transform(lambda x: x.div(x.iloc[0]).subtract(1))
  
    quarterly_all['pre_crisis'] = 0
    crisis = pd.to_datetime('2009-10-1')
    quarterly_all.loc[quarterly_all['date'] < crisis, 'pre_crisis'] = 1

    quarterly_all['sin_month'] = np.sin((quarterly_all['date'].dt.month/6) * np.pi)
    quarterly_all['cos_month'] = np.cos((quarterly_all['date'].dt.month/6) * np.pi)
    
    quarterly_all["current_prepay"] = quarterly_all["previous_UPB"] - quarterly_all["current_UPB"]
    quarterly_all["origin_interest"] = quarterly_all["origin_interest"] / (100 * 12)
    quarterly_all["scheduled_payment"] = (((quarterly_all["origin_interest"] * quarterly_all["origin_UPB"]) / 
                               (1 - np.power(1 + quarterly_all["origin_interest"], -quarterly_all["origin_term"]))) - 
                               (quarterly_all["origin_interest"] * quarterly_all["previous_UPB"]))
    
    quarterly_all["Payment"] = quarterly_all.apply(lambda row : response_var(row), axis=1)
  
    return quarterly_all

## (4) Data Preparing

In [ ]:
direc_output = '/content/drive/MyDrive/Colab Notebooks/Machine Learning/Machine_Learning_Project/Cleaned Data/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
years = np.range(1999, 2022)

for year in years:
  for quarter in tqdm(range(1, 4)):
    data = create_quarterly(year, quarter)
    data.to_csv(direc_output + str(year) + "Q" + str(quarter) +'.csv') 